In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.67.1"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.67.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel CreateKernel()
{
    var builder = Kernel.CreateBuilder();

    var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    var kernel = builder.Build();

    return kernel;
}

var kernel = CreateKernel();

Installed Packages Microsoft.SemanticKernel, 1.67.1 Microsoft.SemanticKernel.Agents.Core, 1.67.1

# Adding functions from a type

In [2]:
using System.ComponentModel;
class KuberenetesSkills
{
    [Description("Runs kubectl commands, just provide the arguments to the kubectl and it will run it and return the result as string")]
    [KernelFunction()]
    public async Task<string> RunKubectl(string kubectlArguements)
    {
        Console.WriteLine($"{kubectlArguements}");
        return "Not implemented";
    }
}

In [3]:
kernel.Plugins.AddFromType<KuberenetesSkills>();

In [4]:
PromptExecutionSettings settings = new() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };

In [5]:
using Microsoft.SemanticKernel.Agents;

var k8sAgent = new ChatCompletionAgent
{
    Name = "KubernetesAgent",
    Instructions = 
        """
        You are an expert in Kubernetes.
        Answer questions about Kubernetes ONLY!.
        """,
    Description = "An expert in Kubernetes",
    Kernel = kernel,


    // Instruct SK to allow the function calling
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

In [6]:

var response = k8sAgent.InvokeAsync("run a new pod in my k8s cluster from image 'myimage_name:latest'");

await foreach (var item in response)
{
    Console.WriteLine(item.Message);
}

run new-pod --image=myimage_name:latest
It seems the functionality to directly run a new pod using `kubectl` is not implemented in this interface. However, you can try running the given command directly on your terminal:

```bash
kubectl run new-pod --image=myimage_name:latest
```

This will create a new pod using the `myimage_name:latest` image. Let me know if you need help with anything else!


# Adding functions from an object

In [7]:
public sealed class TimePlugin
{
    private readonly TimeProvider _timeProvider;

    public TimePlugin(TimeProvider timeProvider)
    {
        this._timeProvider = timeProvider;
    }

    [KernelFunction]
    [Description("Get the date of today Iin UTC as a string")]
    public string GetTodayAsString()
    {
        DateTimeOffset dateTime = this._timeProvider.GetUtcNow();        

        return dateTime.ToString();
    }
}

In [8]:
kernel.ImportPluginFromObject(new TimePlugin(TimeProvider.System));

In [9]:
using Microsoft.SemanticKernel.Agents;

var calendarAgent = new ChatCompletionAgent
{
    Name = "CalendarAgent",
    Instructions = 
        """
        You are an agent that helps with scheduling meeting and manging the user calendar.
        """,
    Description = "An user calendar assistent",
    Kernel = kernel,


    // Instruct SK to allow the function calling
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

In [10]:
var response = k8sAgent.InvokeAsync("what's the date today?");

await foreach (var item in response)
{
    Console.WriteLine(item.Message);
}

Today's date is November 16, 2025.


# MCP Client

In [11]:
#r "nuget: System.Net.ServerSentEvents, 10.0.0-preview.4.25258.110"

#r "nuget: ModelContextProtocol, 0.3.0-preview.3"

Installed Packages ModelContextProtocol, 0.3.0-preview.3 System.Net.ServerSentEvents, 10.0.0-preview.4.25258.110

In [12]:
using ModelContextProtocol.Client;

In [ ]:
// https://github.com/modelcontextprotocol/servers-archived/tree/main/src/github

var mcpClient = await McpClient.CreateAsync(
    new StdioClientTransport(new()
        {
            Name = "MCPServer",
            Command = "npx",
            Arguments = ["-y", "@modelcontextprotocol/server-github"],

            // EnvironmentVariables = new Dictionary<string, string>
            // {
            //     ["GITHUB_PERSONAL_ACCESS_TOKEN":] = "XXXXXXXX"             
            // }
        }));

// Retrieve the list of tools available on the MCP server
var tools = await mcpClient.ListToolsAsync().ConfigureAwait(false);

In [14]:
#pragma warning disable SKEXP0001

kernel.Plugins.AddFromFunctions("github", tools.Select(aiFunction => aiFunction.AsKernelFunction()));

In [15]:
using Microsoft.SemanticKernel.Agents;

var githubAgent = new ChatCompletionAgent
{
    Name = "GithubAgent",
    Instructions = 
        """
        You are an agent that helps questions about github
        """,
    Description = "An github assistent",
    Kernel = kernel,


    // Instruct SK to allow the function calling
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

In [16]:
var response = githubAgent.InvokeAsync("list the latest commits of the semantic-kernel repo?");

await foreach (var item in response)
{
    Console.WriteLine(item.Message);
}

Here are some of the latest commits in the [semantic-kernel repository](https://github.com/microsoft/semantic-kernel):

1. **Commit:** [de20575a](https://github.com/microsoft/semantic-kernel/commit/de20575aee70ddb31d4b58bbd7b837101ed82cfb)
   - **Message:** "Fixes to Get/Delete for Guid keys (#13353)"
   - **Author:** Adam Sitnik
   - **Date:** 2025-11-13T11:05:42Z

2. **Commit:** [849cff0b](https://github.com/microsoft/semantic-kernel/commit/849cff0b5048483ca5138dabdeeea9ee22c717ac)
   - **Message:** "Python: Bump Python version to 1.38.0 for a release. (#13354)"
   - **Author:** Evan Mattson
   - **Date:** 2025-11-11T10:18:05Z

3. **Commit:** [145ad9bf](https://github.com/microsoft/semantic-kernel/commit/145ad9bfd2f9c31285d23918c9b9ede54d7fa68a)
   - **Message:** "Python: Upgrade Onnx Connector to use 0.9.0 (#13162)"
   - **Author:** Nico Möller
   - **Date:** 2025-11-11T08:44:52Z

4. **Commit:** [b1ecee2d](https://github.com/microsoft/semantic-kernel/commit/b1ecee2d0e513963a3aef99bb

MCP Remote server

In [17]:
//Trick to resolve the issue with DLL loading
{
    var t=typeof(System.Net.ServerSentEvents.SseFormatter);
    Console.WriteLine(t.Assembly.FullName);

    AppDomain.CurrentDomain.AssemblyResolve += (sender, args) =>
    {
        
        //Console.WriteLine($"AssemblyResolve {args.Name}");

        // If the missing assembly matches something you're expecting
        if (args.Name.StartsWith("System.Net.ServerSentEvents"))
        {

            return t.Assembly;
        }

        return null; // Let the runtime continue trying otherwise
    };
}


System.Net.ServerSentEvents, Version=10.0.0.0, Culture=neutral, PublicKeyToken=cc7b13ffcd2ddd51


In [ ]:
var mcpClient = await McpClient.CreateAsync(
    new HttpClientTransport(
        new HttpClientTransportOptions()
        { 
            Endpoint = new Uri("https://learn.microsoft.com/api/mcp") 
        }));

// Retrieve the list of tools available on the MCP server
var tools = await mcpClient.ListToolsAsync().ConfigureAwait(false);

In [19]:
#pragma warning disable SKEXP0001

kernel.Plugins.AddFromFunctions("MSDocs", tools.Select(aiFunction => aiFunction.AsKernelFunction()));

In [20]:
using Microsoft.SemanticKernel.Agents;

var microsoftDevAgent = new ChatCompletionAgent
{
    Name = "MSDevAgent",
    Instructions = 
        """
        You are a software developer that profession in Microsoft technologies
        """,
    Description = "A Software Dev",
    Kernel = kernel,


    // Instruct SK to allow the function calling
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

var response = githubAgent.InvokeAsync("which nuget do you need to add to create chat completion agent with semantic kernel?");

await foreach (var item in response)
{
    Console.WriteLine(item.Message);
}

To create a Chat Completion agent with Semantic Kernel, you typically need to install the following NuGet packages:

1. **[Microsoft.SemanticKernel](https://www.nuget.org/packages/Microsoft.SemanticKernel)**: This contains the core Semantic Kernel libraries required for working with the Agent Framework.

2. **[Microsoft.SemanticKernel.Agents.Abstractions](https://www.nuget.org/packages/Microsoft.SemanticKernel.Agents.Abstractions)**: Defines the core agent abstractions. Note that this is automatically included in the following optional packages:
   - **[Microsoft.SemanticKernel.Agents.Core](https://www.nuget.org/packages/Microsoft.SemanticKernel.Agents.Core)**: Includes the `ChatCompletionAgent` type needed for chat interactions.
   - **[Microsoft.SemanticKernel.Agents.OpenAI](https://www.nuget.org/packages/Microsoft.SemanticKernel.Agents.OpenAI)**: Provides the ability to use OpenAI Assistant API with the `OpenAIAssistantAgent`.

3. **[Microsoft.SemanticKernel.Agents.Orchestration](ht

# Adding your dotnet tool as MCP Server

MCP Server using stdio: [..\src\SKCodeAssistent\FirstMCPServer](..\src\SKCodeAssistent\FirstMCPServer\Program.cs)

In [21]:
Console.WriteLine(Environment.CurrentDirectory);

c:\Users\tamirdresher\source\repos\creating-ai-agents-with-csharp\notebooks


In [ ]:
#pragma warning disable SKEXP0001

var kernel = CreateKernel();

var mcpClient = await McpClient.CreateAsync(
    new StdioClientTransport(new()
        {
            Name = "MyFirstLocalMCP",
            Command = "dotnet",
            Arguments = ["run", "--project", "..\\src\\SKCodeAssistent\\FirstMCPServer\\FirstMCPServer.csproj"],
        }));

// Retrieve the list of tools available on the MCP server
var tools = await mcpClient.ListToolsAsync().ConfigureAwait(false);

kernel.Plugins.AddFromFunctions("myMcp", tools.Select(aiFunction => aiFunction.AsKernelFunction()));


PromptExecutionSettings settings = new() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };
var response = await kernel.InvokePromptAsync("echo my name: Tamir Dresher", new KernelArguments(settings));

Console.WriteLine(response);

Here you go:

- Echo: **Hello from C#: Tamir Dresher**
- Reverse Echo: **rehserD rimaT**


# Extra: MCP Server

First, install the base templates

```shell
dotnet new install Microsoft.Extensions.AI.Templates
```

then, create you MCP Server project

```shell
dotnet new mcpserver -n SampleMcpServer
cd SampleMcpServer
dotnet build
```

This will create a starting project which uses the `ModelContextProtocol` C# SDK

In [23]:
#r "nuget: ModelContextProtocol, 0.3.0-preview.3"

Installed Packages ModelContextProtocol, 0.3.0-preview.3

## Add you tools by decorating your class methods with the `[McpServerTool]` attribute

In [24]:
using ModelContextProtocol.Server;

[McpServerTool]
[Description("Describes random weather in the provided city.")]
public string GetCityWeather(
    [Description("Name of the city to return weather for")] string city)
{
    var weather = "balmy,rainy,stormy";    

    var weatherChoices = weather.Split(",");
    var selectedWeatherIndex =  Random.Shared.Next(0, weatherChoices.Length);

    return $"The weather in {city} is {weatherChoices[selectedWeatherIndex]}.";
}

## Publishing to nuget
follow the steps here:
https://devblogs.microsoft.com/dotnet/mcp-server-dotnet-nuget-quickstart/#📋-configuring-for-nuget-publication

Nuget is able to filter for packages with MCP Servers
![image.png](./img/nuget-mcp-search.png)

## Consuming a nuget based MCP Servers

From net10, the SDK supports the `dnx` command (like npx for dotnet)

Simply put this in your MCP Client config

```json
{
  "inputs": [],
  "servers": {
    "mcpserver.everything.stdio": {
      "type": "stdio",
      "command": "dnx",
      "args": ["YOUR_NUGET_PACKAGE@0.4.0", "--yes"],
      "env": {}
    }
  }
}
```

![image.png](./img/nuget-mcp-use-instructions.png)